# Purpose of this notebook
There are regions in the Allen and Princeton Mouse Brain ontologies that do not have any voxels associated with them in the annotation volume. In the "search by region name" toolbar in Neuroglancer, the names of brain regions in a provided atlas are displayed. The names are derived from the ontology and therefore there are some that have no corresponding region in the segmentation volume that represents the annotation volume. In this notebook, we remove these regions from the file that Neuroglancer uses to map region to region name so that they no longer will appear in Neuroglancer.


In [31]:
import os,csv
import pandas as pd
import numpy as np
import tifffile

# Point to the csv files for both atlases containing the region names and voxel counts 
csv_file_allen = '/home/ahoag/ngdemo/data/allen_id_table_w_voxel_counts.csv'
annotation_volume_allen = '/jukebox/LightSheetData/atlas/allen_atlas/annotation_template_25_sagittal.tif'

In [45]:
df_allen = pd.read_csv(csv_file_allen).sort_values("id")
df_allen

,row,name,acronym,id,atlas_id,parent_structure_id,parent_name,parent_acronym,voxels_in_structure
775,775,"Tuberomammillary nucleus, ventral part",TMv,1,424.0,557.0,Tuberomammillary nucleus,TM,6931
78,78,"Primary somatosensory area, mouth, layer 6b",SSp-m6b,2,990.0,345.0,"Primary somatosensory area, mouth",SSp-m,7339
1316,1316,secondary fissure,sec,3,707.0,1040.0,grooves of the cerebellar cortex,grv of CBX,0
812,812,Inferior colliculus,IC,4,141.0,339.0,"Midbrain, sensory related",MBsen,0
1201,1201,internal capsule,int,6,566.0,784.0,corticospinal tract,cst,134366
...,...,...,...,...,...,...,...,...,...
875,875,"Interpeduncular nucleus, intermediate",IPI,607344850,NaN,100.0,NaN,NaN,0
876,876,"Interpeduncular nucleus, dorsomedial",IPDM,607344854,NaN,100.0,NaN,NaN,0
877,877,"Interpeduncular nucleus, dorsolateral",IPDL,607344858,NaN,100.0,NaN,NaN,0
878,878,"Interpeduncular nucleus, rostrolateral",IPRL,607344862,NaN,100.0,NaN,NaN,0


Look at the rendering of this volume in Neuroglancer, the "voxels_in_structure" column is not accurate. There are regions that say they have voxels that do not. The only safe way to restructure the list is to go through the annotation volume and see which IDs are actually in the volume. Then use that list to trim off the entries in the dataframe. 

In [33]:
annotation_vol = tifffile.imread(annotation_volume_allen)

In [34]:
annotation_ids = np.unique(annotation_vol)
annotation_ids

array([0.0000000e+00, 1.0000000e+00, 2.0000000e+00, 6.0000000e+00,
       7.0000000e+00, 9.0000000e+00, 1.0000000e+01, 1.2000000e+01,
       1.5000000e+01, 1.7000000e+01, 1.9000000e+01, 2.0000000e+01,
       2.3000000e+01, 2.6000000e+01, 2.7000000e+01, 2.8000000e+01,
       3.0000000e+01, 3.3000000e+01, 3.5000000e+01, 3.6000000e+01,
       3.8000000e+01, 4.1000000e+01, 4.2000000e+01, 5.0000000e+01,
       5.2000000e+01, 5.4000000e+01, 5.6000000e+01, 5.8000000e+01,
       5.9000000e+01, 6.2000000e+01, 6.3000000e+01, 6.4000000e+01,
       6.6000000e+01, 6.7000000e+01, 6.8000000e+01, 7.2000000e+01,
       7.4000000e+01, 7.5000000e+01, 7.8000000e+01, 8.1000000e+01,
       8.3000000e+01, 8.4000000e+01, 8.8000000e+01, 9.1000000e+01,
       9.3000000e+01, 9.6000000e+01, 9.7000000e+01, 9.8000000e+01,
       1.0000000e+02, 1.0100000e+02, 1.0200000e+02, 1.0500000e+02,
       1.0600000e+02, 1.0800000e+02, 1.1300000e+02, 1.1400000e+02,
       1.1500000e+02, 1.1700000e+02, 1.1800000e+02, 1.2000000e

In [46]:
df_ids = df_allen['id']
df_ids

775             1
78              2
1316            3
812             4
1201            6
          ...    
875     607344850
876     607344854
877     607344858
878     607344862
842     614454277
Name: id, Length: 1327, dtype: int64

In [47]:
ids_in_volume_mask = df_ids.isin(annotation_ids)

In [48]:
df_allen_in_volume = df_allen[ids_in_volume_mask]
df_allen_in_volume

,row,name,acronym,id,atlas_id,parent_structure_id,parent_name,parent_acronym,voxels_in_structure
775,775,"Tuberomammillary nucleus, ventral part",TMv,1,424.0,557.0,Tuberomammillary nucleus,TM,6931
78,78,"Primary somatosensory area, mouth, layer 6b",SSp-m6b,2,990.0,345.0,"Primary somatosensory area, mouth",SSp-m,7339
1201,1201,internal capsule,int,6,566.0,784.0,corticospinal tract,cst,134366
889,889,Principal sensory nucleus of the trigeminal,PSV,7,283.0,1132.0,"Pons, sensory related",P-sen,70327
91,91,"Primary somatosensory area, trunk, layer 6a",SSp-tr6a,9,991.0,361.0,"Primary somatosensory area, trunk",SSp-tr,14877
...,...,...,...,...,...,...,...,...,...
475,475,"Dentate gyrus, polymorph layer",DG-po,10704,NaN,726.0,NaN,NaN,43800
359,359,"Rostrolateral area, layer 6b",VISrl6b,312782624,NaN,417.0,NaN,NaN,76472
550,550,"Prosubiculum, ventral part, molecular layer",ProSv-m,484682496,NaN,484682492.0,NaN,NaN,89266
1260,1260,commissural branch of stria terminalis,stc,484682528,NaN,301.0,NaN,NaN,341505


Can see that only about half of the original structures are actually in the annotation volume. Now let's use this list of IDs as the hardcoded PMA and make 

In [49]:
IDs=df_allen_in_volume['id'].values
names=df_allen_in_volume['name'].values
acronyms = df_allen_in_volume['acronym'].values

In [50]:
for ii in range(len(IDs)):
    ID=IDs[ii]
    name=names[ii]
    acronym=acronyms[ii]
    line = f'[{ID}, "{ID}: {acronym}: {name}"],'
    print(line)

[1, "1: TMv: Tuberomammillary nucleus, ventral part"],
[2, "2: SSp-m6b: Primary somatosensory area, mouth, layer 6b"],
[6, "6: int: internal capsule"],
[7, "7: PSV: Principal sensory nucleus of the trigeminal"],
[9, "9: SSp-tr6a: Primary somatosensory area, trunk, layer 6a"],
[10, "10: SCig: Superior colliculus, motor related, intermediate gray layer"],
[12, "12: IF: Interfascicular nucleus raphe"],
[15, "15: PT: Parataenial nucleus"],
[17, "17: SCiw: Superior colliculus, motor related, intermediate white layer"],
[19, "19: IG: Induseum griseum"],
[20, "20: ENTl2: Entorhinal area, lateral part, layer 2"],
[23, "23: AAA: Anterior amygdalar area"],
[26, "26: SCdg: Superior colliculus, motor related, deep gray layer"],
[27, "27: IGL: Intergeniculate leaflet of the lateral geniculate complex"],
[28, "28: ENTl6a: Entorhinal area, lateral part, layer 6a"],
[30, "30: PVa: Periventricular hypothalamic nucleus, anterior part"],
[33, "33: VISp6a: Primary visual area, layer 6a"],
[35, "35: III: O

## Step 1: Convert the CSV annotation file into JSON file that Neuroglancer can read

In [2]:
def make_annotation_json_file(csv_input_file):
    """ Take a csv file of the format of labels_v3.csv 
    containing the MRI atlas labels and converts it to a 
    JSON file so that Neuroglancer can read it in
    
    The JSON file will be saved with the same filename as 
    the CSV file but with .json extension instead of .csv
    """
    annotation_json_file = csv_input_file.replace('.csv','.json')

    with open(csv_input_file,'r') as infile:
        reader = csv.reader(infile)
        next(reader) # skips the header since we just want the data and we already know what the order of the columns is
        annotation_dict = {row[2]:row[1] for row in reader}
        annotation_dict_list = [{f"regionId":key,"regionName":f"{key}: {annotation_dict[key]}"} for key in annotation_dict.keys()]
    with open(annotation_json_file,'w') as outfile:
        outfile.write(
            '[' +
            ',\n'.join(json.dumps(i) for i in annotation_dict_list) +
            ']\n')
    # Now move the annotation file to the layer directory
    annotation_rel_path = annotation_json_file.split('/')[-1]
    dst = os.path.join(layer_dir,annotation_rel_path)
    os.rename(annotation_json_file,dst)
    print(f"Wrote file {dst}")
    return dst

In [3]:
# run the function to convert to JSON
annotation_json_fullpath=make_annotation_json_file(annotation_csv_file)

Wrote file /home/ahoag/progs/brodylab/atlas/my_custom_atlas1/labels_v3.json


## Step 2: Write the instructions ("info") file that will tell Neuroglancer about your annotation volume and the annotation label file you just created
The info file is a required file for the precomputed data format. It is a JSON file containing things like the shape and physical resolution of your volume, and it will also be where we tell it to read in our labels for the different anatomical regions

In [4]:
def make_info_file(resolution_xyz,volume_size_xyz,annotation_filename,layer_dir):
    """ Make an JSON-formatted file called the "info" file
    for use with the precomputed data format. 
    Precomputed is one of the formats that Neuroglancer can read in.  
    --- parameters ---
    resolution_xyz:      A tuple representing the size of the pixels (dx,dy,dz) 
                         in nanometers, e.g. (20000,20000,5000) for 20 micron x 20 micron x 5 micron
    
    volume_size_xyz:     A tuple representing the number of pixels in each dimension (Nx,Ny,Nz)
    
    annotation_filename: The relative path (from layer_dir) to the JSON-formatted file
                         that we created above containing the labels of the different regions.
                         
    layer_dir:           The directory where the precomputed data will be
                         saved
    """
    info = CloudVolume.create_new_info(
        num_channels = 1,
        layer_type = 'segmentation', # 'image' or 'segmentation'
        data_type = 'uint16', # 32 not necessary for atlases unless you have > 2^(32)-1 labels  
        encoding = 'raw', # other options: 'jpeg', 'compressed_segmentation' (req. uint32 or uint64)
        resolution = resolution_xyz, # X,Y,Z values in nanometers, 40 microns in each dim
        voxel_offset = [ 0, 0, 0 ], # values X,Y,Z values in voxels
        chunk_size = [ 1024, 1024, 1 ], # rechunk of image X,Y,Z in voxels.
        volume_size = volume_size_xyz, # X,Y,Z size in voxels
    )

    vol = CloudVolume(f'file://{layer_dir}', info=info)
    vol.provenance.description = "A test info file" # can change this if you want a description
    vol.provenance.owners = [''] # list of contact email addresses
    # Saves the info and provenance files for the first time
    vol.commit_info() # generates file://bucket/dataset/layer/info json file
    vol.commit_provenance() # generates file://bucket/dataset/layer/provenance json file
    # add a key for the custom atlas annotations to the info file and overwrite it 
    info_dict = vol.info
    info_dict['atlas_path'] = annotation_filename
    info_filename = '/'.join(vol.info_cloudpath.split('/')[2:]) 
    with open(info_filename,'w') as outfile:
        json.dump(info_dict,outfile,sort_keys=True,indent=2)
    print("Created CloudVolume info file: ",vol.info_cloudpath)

    return vol

In [5]:
## Make the info file

# The MRI atlas has 39 micron isotropic resolution - we need nanometers though
resolution_xyz = (39000,39000,39000)
# Load the MRI annotation volume into memory and get its shape 
annotation_vol = np.array(sitk.GetArrayFromImage(
    sitk.ReadImage(custom_annotation_vol_path)),dtype=np.uint16,order='F')
z_dim,y_dim,x_dim = annotation_vol.shape
volume_size_xyz = (x_dim,y_dim,z_dim)

# Now get the relative path to the annotation file
annotation_json_relpath = annotation_json_fullpath.split('/')[-1]

# Write the info file
vol = make_info_file(
    resolution_xyz=resolution_xyz,
    volume_size_xyz=volume_size_xyz,
    annotation_filename=annotation_json_relpath,
    layer_dir=layer_dir)


Created CloudVolume info file:  file:///home/ahoag/progs/brodylab/atlas/my_custom_atlas1/info


# Step 3: Convert annotation volume to precomputed data format
First we create a directory (the "progress_dir") at the same folder level as the layer directory to keep track of the progress of the conversion. 
All the conversion does is copy the numpy array representing the 3d volume to a new object "vol". This is done one plane at a time (although it is parallelized). As each plane is converted, an empty file is created in the progress_dir with the name of the plane. By the end of the conversion, there should be as many files in this progress_dir as there are z planes. 

In [6]:
layer_name = layer_dir.split('/')[-1]
parent_dir = '/'.join(layer_dir.split('/')[:-1])
# print(parent_dir)
progress_dir = mkdir(parent_dir+ f'/progress_{layer_name}') # unlike os.mkdir doesn't crash on prexisting 
print(f"created directory: {progress_dir}")

created directory: /home/ahoag/progs/brodylab/atlas/progress_my_custom_atlas1


In [7]:
def process_slice(z):
    """ This function copies a 2d image slice from the atlas volume
    to the cloudvolume object, vol. We will run this in parallel over 
    all z planes
    ---parameters---
    z:    An integer representing the 0-indexed z plane to be converted
    """
    if os.path.exists(os.path.join(progress_dir, str(z))):
        print(f"Slice {z} already processed, skipping ")
        return
    if z >= z_dim: # z is zero indexed and runs from 0-(z_dim-1)
        print("Index {z} >= z_dim of volume, skipping")
        return
    print('Processing slice z=',z)
    array = annotation_vol[z].reshape((1,y_dim,x_dim)).T
    vol[:,:, z] = array
    touch(os.path.join(progress_dir, str(z)))
    return "success"


In [8]:
# Run the conversion in parallel. It's not a huge amount of processing but the more cores the better

# First figure out if there are any planes that have already been converted 
# by checking the progress dir
done_files = set([ int(z) for z in os.listdir(progress_dir) ])
all_files = set(range(vol.bounds.minpt.z, vol.bounds.maxpt.z))
# Figure out the ones we still need to convert 
to_upload = [ int(z) for z in list(all_files.difference(done_files)) ]
to_upload.sort()
print(f"Have {len(to_upload)} planes to upload")
with ProcessPoolExecutor(max_workers=cpus_to_use) as executor:
    for result in executor.map(process_slice,to_upload):
        try:
            print(result)
        except Exception as exc:
            print(f'generated an exception: {exc}')

Have 455 planes to upload
Processing slice z= 0
Processing slice z= 1
Processing slice z= 6
Processing slice z= 2
Processing slice z= 4
Processing slice z= 3
Processing slice z= 5
Processing slice z= 7


Uploading: 100%|██████████| 1/1 [00:00<00:00, 57.06it/s]

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Processing slice z= 8



Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Processing slice z= 9


...

Processing slice z= 453



Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Processing slice z= 454
success
success
success
success
success
success



Uploading: 100%|██████████| 1/1 [00:00<00:00, 81.62it/s]

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]


success
success
success
success
success
success
success
success


# Step 4: Host the precomputed data on your machine so that Neuroglancer can see it
This step is really easy! Note: Exectuing the cell below will cause your jupyter notebook to hang (expected). This is the last cell you need to run in the notebook, but if for some reason you want to use the same notebook for something else then run the code in the cell below elsewhere (e.g. a new terminal window)

In [ ]:
vol = CloudVolume(f'file://{layer_dir}')
vol.viewer(port=1338)

Neuroglancer server listening to http://localhost:1338


127.0.0.1 - - [28/Apr/2020 16:17:31] "GET /info HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:32] "GET /labels_v3.json HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:32] "GET /39000_39000_39000/0-302_0-774_227-228 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:32] "GET /39000_39000_39000/0-302_0-774_228-229 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:32] "GET /39000_39000_39000/0-302_0-774_229-230 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:32] "GET /39000_39000_39000/0-302_0-774_226-227 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:32] "GET /39000_39000_39000/0-302_0-774_225-226 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:32] "GET /39000_39000_39000/0-302_0-774_230-231 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:32] "GET /39000_39000_39000/0-302_0-774_224-225 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:32] "GET /39000_39000_39000/0-302_0-774_231-232 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:32] "GET /39000_39000_39000/0-302_0-774_223-224 HTTP/

127.0.0.1 - - [28/Apr/2020 16:17:33] "GET /39000_39000_39000/0-302_0-774_272-273 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:33] "GET /39000_39000_39000/0-302_0-774_273-274 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:33] "GET /39000_39000_39000/0-302_0-774_182-183 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:33] "GET /39000_39000_39000/0-302_0-774_181-182 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:33] "GET /39000_39000_39000/0-302_0-774_274-275 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:33] "GET /39000_39000_39000/0-302_0-774_275-276 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:33] "GET /39000_39000_39000/0-302_0-774_180-181 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:33] "GET /39000_39000_39000/0-302_0-774_179-180 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:33] "GET /39000_39000_39000/0-302_0-774_276-277 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:33] "GET /39000_39000_39000/0-302_0-774_277-278 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:

127.0.0.1 - - [28/Apr/2020 16:17:34] "GET /39000_39000_39000/0-302_0-774_136-137 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:34] "GET /39000_39000_39000/0-302_0-774_135-136 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:34] "GET /39000_39000_39000/0-302_0-774_320-321 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:34] "GET /39000_39000_39000/0-302_0-774_321-322 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:34] "GET /39000_39000_39000/0-302_0-774_134-135 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:34] "GET /39000_39000_39000/0-302_0-774_133-134 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:34] "GET /39000_39000_39000/0-302_0-774_322-323 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:34] "GET /39000_39000_39000/0-302_0-774_323-324 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:34] "GET /39000_39000_39000/0-302_0-774_132-133 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:17:34] "GET /39000_39000_39000/0-302_0-774_131-132 HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 16:

# Step 5: View your custom volume and labels in Neuroglancer!
Step 4 hosts your data via http on port 1338 of your local machine. When you run Neuroglancer in your browser, you can tell it to look for data hosted at a particular port. To do this, open up the Braincogs Neuroglancer client: [https://braincogs00.pni.princeton.edu/nglancer_viewer](https://braincogs00.pni.princeton.edu/nglancer_viewer) (you must be using a Princeton VPN) and then click the "+" in the upper left hand corner of the screen once the black screen loads. To load in your data, type the following into the source text box:<br>
> precomputed://http://localhost:1338 <br>

Then hit tab and name your layer if you'd like. Hit enter or the "add layer" button and your layer should load into Neuroglancer. Hopefully the labels you added should be showing up in the bottom left when you hover over a region. If not, let me know!

Note: If you want to load in multiple layers, you will need to host multiple layers. To do this, just host them at different ports (ports 133X, where X:0-9 are usually not used by anything else on linux in my experience). By default they will overlap with each other, which is not always the most convenient thing. You can drag and drop the layers around so they sit next to each other but it is a little finicky.  I'm working on some videos to show how this is done, but if you run into trouble let me know.